In [1]:
from dotenv import load_dotenv
import os 

load_dotenv()

DB_NAME = os.getenv("dbname")
USER = os.getenv("user")
PASSWORD = os.getenv("password")
HOST = os.getenv("host")
PORT = os.getenv("port")
openai_api_key = os.getenv("OPENAI_API_KEY")

In [2]:
print(HOST)

aws-0-ap-south-1.pooler.supabase.com


In [3]:
import json
import os
import psycopg2
from dotenv import load_dotenv

load_dotenv()

DB_NAME = os.getenv("dbname")
USER = os.getenv("user")
PASSWORD = os.getenv("password")
HOST = os.getenv("host")
PORT = os.getenv("port")

def connect_to_superbase(db_name,user,password,host,port):
    connection = None
    try:
        connection = psycopg2.connect(
            database = db_name,
            user = user,
            password = password,
            host = host,
            port = port 
        )
        print(f"Connect to postgress DB({db_name}) is succesful.")
        return connection
    except Exception as e:
        print(f"Error occured while creating connection to DB: {db_name}, Error: {str(e)}")
        return connection

conn_online_db = connect_to_superbase(DB_NAME,USER,PASSWORD,HOST,PORT)

def execute_query(connection,query,params=None):
    try:
        cursor = connection.cursor()
        cursor.execute(query,params)
        if query.strip().upper().startswith(('CREATE','INSERT','UPDATE','DELETE')):
            connection.commit()
            if "RETURNING" in query.upper():
                result = cursor.fetchone()[0]
            else:
                result = cursor.rowcount
            return result
        else:
            print("Query Executed")
            return cursor.fetchall()
    except Exception as e:
        print(f"{str(e)}")
        if connection:
            connection.rollback()
        return None
    finally:
        if cursor:
            cursor.close()

Connect to postgress DB(postgres) is succesful.


In [4]:
# Create the resumes table
create_resumes_table = """
CREATE TABLE IF NOT EXISTS resumes (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    original_filename TEXT,
    file_type TEXT,
    raw_text TEXT,
    parsed_json JSONB,
    uploaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

# Create the job_descriptions table
create_jd_table = """
CREATE TABLE IF NOT EXISTS job_descriptions (
    id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    resume_id UUID REFERENCES resumes(id) ON DELETE CASCADE,
    original_filename TEXT,
    raw_text TEXT,
    parsed_json JSONB,
    uploaded_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

# Create the scores table
create_scores_table = """
CREATE TABLE IF NOT EXISTS scores (
    score_id UUID PRIMARY KEY DEFAULT gen_random_uuid(),
    resume_id UUID REFERENCES resumes(id) ON DELETE CASCADE,
    score_breakdown JSONB,
    total_score FLOAT,
    recommendations JSONB,
    created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);
"""

if conn_online_db:
    execute_query(conn_online_db, "CREATE EXTENSION IF NOT EXISTS pgcrypto;")
    execute_query(conn_online_db, create_resumes_table)
    execute_query(conn_online_db, create_jd_table)
    execute_query(conn_online_db, create_scores_table)


In [5]:
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.ui import Console
from autogen_agentchat.base import TaskResult
import os
from dotenv import load_dotenv
load_dotenv()

api_key = os.getenv('GROQ_API_KEY')

model_client = OpenAIChatCompletionClient(
    base_url="https://api.groq.com/openai/v1", 
    model="openai/gpt-oss-120b", 
    api_key=api_key, 
    model_info={
        "family": "groq",
        "vision": False, 
        "function_calling": True,
        "json_output": False
    }
) 

sample_agent = AssistantAgent(
    name="sample_agent",
    model_client=model_client,
    system_message="You are a helpful assistant."
)

result = await sample_agent.run(task=(TextMessage(content="Hello how are you",source="user")))
print(result.messages[-1].content)

c:\Users\mkuma\OneDrive\Desktop\MEOW MEOW 1.0\.venv\Lib\site-packages\autogen_ext\models\openai\_openai_client.py:453: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


Hello! I'm doing well, thank you for asking. How can I assist you today?


In [6]:
import pdfplumber
import docx
import mimetypes


def extract_text_from_file(file_path:str)->str:
    """Extract text from PDF, DOCX, or TXT file without textract."""
    file_type,_ = mimetypes.guess_type(file_path)

    if file_type =='application/pdf':
        with pdfplumber.open(file_path) as pdf:
            text = "\n".join([page.extract_text() for page in pdf.pages])
    elif file_type == "application/vnd.openxmlformats-officedocument.wordprocessingml.document":
        doc_file = docx.Document(file_path)
        text = "\n".join([para.text for para in doc_file.paragraphs])
    elif file_type == "text/plain":
        with open(file_path,"r",encoding="utf-8") as f:
            text = f.read()
    else:
        raise ValueError(f"Unsupported file type: {file_type}")
    
    return file_type,text.strip()


In [7]:
from optparse import Option
from pydantic import BaseModel,Field
from typing import List,Dict,Optional

class ResumeParser(BaseModel):
    """Parse key resume fields from text"""

    file_type : Optional[str] = Field(None,description="File type of the resume.")

    name : Optional[str] = Field(None,description="Full name of the candidate.")

    contact : Optional[Dict[str,Optional[str]]] = Field(
        None,
        description="Contact information with email and phone."
    )
    skills : List[str] = Field(
        default_factory=list,
        description="List of skills mentioned in the resume.",
    )
    experience : List[str] = Field(
        default_factory=list,
        description="List of work experience mentioned in the resume.",
    )
    education : List[str] = Field(
        default_factory=list,
        description="List of education mentioned in the resume.",
    )
    certifications : List[str] = Field(
        default_factory=list,
        description="List of certifications mentioned in the resume.",
    )

In [8]:
extracted_text = extract_text_from_file("resume.txt")
print(extracted_text)

('text/plain', 'John Doe\nEmail: john.doe@example.com\nPhone: +1 987 654 3210\n\nProfessional Summary:\nHighly motivated software engineer with 4 years of experience in backend and cloud technologies.\n\nSkills:\nPython, SQL, Java, Machine Learning, AWS, Docker, Kubernetes\n\nExperience:\nSoftware Engineer - ABC Tech (2018 - 2020)\nSenior Software Engineer - XYZ Solutions (2020 - Present)\n\nEducation:\nB.Tech Computer Science - University of Example (2014 - 2018)\n\nCertifications:\nAWS Certified Developer')


In [9]:
openai_api_key = os.getenv('OPENAI_API_KEY')
json_model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_key=openai_api_key,
    response_format=ResumeParser                                                                                           
)
extracted_text_json = AssistantAgent(
    name="json_extractor",
    model_client= json_model_client,
    system_message="""You are a resume parser agent when you are given a resume text, you will extract the information from the resume and return it in a json format. In the extracted text there is also file_type written as starting""")

In [10]:
result = await extracted_text_json.run(task=(TextMessage(content=f"Extract text from the resume\n\n\n {extracted_text}",source="user")))
print(result.messages[-1].content)

{"file_type":"text/plain","name":"John Doe","contact":{"email":"john.doe@example.com","phone":"+1 987 654 3210"},"skills":["Python","SQL","Java","Machine Learning","AWS","Docker","Kubernetes"],"experience":["Software Engineer - ABC Tech (2018 - 2020)","Senior Software Engineer - XYZ Solutions (2020 - Present)"],"education":["B.Tech Computer Science - University of Example (2014 - 2018)"],"certifications":["AWS Certified Developer"]}


In [11]:
content = result.messages[-1].content

In [12]:
import json
parsed_data = json.loads(content)
print(parsed_data)
parsed_data['file_type']

{'file_type': 'text/plain', 'name': 'John Doe', 'contact': {'email': 'john.doe@example.com', 'phone': '+1 987 654 3210'}, 'skills': ['Python', 'SQL', 'Java', 'Machine Learning', 'AWS', 'Docker', 'Kubernetes'], 'experience': ['Software Engineer - ABC Tech (2018 - 2020)', 'Senior Software Engineer - XYZ Solutions (2020 - Present)'], 'education': ['B.Tech Computer Science - University of Example (2014 - 2018)'], 'certifications': ['AWS Certified Developer']}


'text/plain'

In [62]:
ids = []
def save_resume_to_db(conn,parsed_string):
    query = """
    INSERT INTO resumes (file_type, raw_text, parsed_json)
    VALUES (%s, %s, %s)
    RETURNING id;
    """
    params = (
        parsed_string['file_type'],
        extracted_text,
        json.dumps(parsed_string),
    )
    try:
        if conn_online_db:
            resume_id = execute_query(conn, query, params)
            ids.append(resume_id)
            return f"Resume saved to database successfully and id is {resume_id}"
        else:
            raise Exception("Connection to online database is not established")
    except Exception as e:
        raise Exception(f"Failed to save resume to database: {str(e)}")

In [14]:
save_resume_to_db(conn_online_db,parsed_data)

'Resume saved to database successfully'

In [15]:
import requests
import os
from dotenv import load_dotenv
load_dotenv()

JSEARCH_API = os.getenv('JSEARCH_API_KEY')

def JSearchAPI(query:str):
    """The tool searches for job based on the query"""
    url = "https://jsearch.p.rapidapi.com/search"
    headers = {
    'x-rapidapi-key':JSEARCH_API,
    'x-rapidapi-host': "jsearch.p.rapidapi.com"
    }

    params = {
        "query":query,
        "country":"in",
        "num_pages":1,
        "page":1,
    }

    response = requests.get(url,headers=headers,params=params)
    data = response.json()

    jobs = []
    for job in data.get("data", [])[:5]:
        jobs.append({
            "role": job.get("job_title"),
            "company": job.get("employer_name"),
            "employment_type": job.get("job_employment_type"),
            "job_location": job.get("job_location"),
            "job_salary": f"{job.get('job_min_salary')} - {job.get('job_max_salary')} {job.get('job_salary_period') or ''}".strip(),
            "job_description": job.get("job_description", ""),
            "job_apply_link": job.get("job_apply_link", "")
        })
    return jobs

jobs = JSearchAPI("python developer")
print(jobs)

[{'role': 'Lead Software Engineer - Python', 'company': 'EPAM Systems', 'employment_type': 'Full–time', 'job_location': 'India', 'job_salary': 'None - None', 'job_description': 'We are seeking a highly experienced Lead Software Engineer with expertise in Python to lead development efforts, influence technical direction, and deliver robust, scalable solutions.\n\nThe ideal candidate will bring proven expertise in Python, database systems, and modern software development practices to drive the success of critical projects.\n\nResponsibilities\n• Collaborate with stakeholders to gather requirements, create technical designs, and align solutions with business goals\n• Lead the development of high-quality, scalable, and maintainable software systems\n• Conduct technical reviews, including code reviews, to ensure adherence to best practices, coding standards, and performance benchmarks\n• Coordinate with cross-functional teams to ensure successful implementation of features and solutions\n• 

In [16]:
from autogen_core.tools import FunctionTool
JobFinder = FunctionTool(func=JSearchAPI,name="JobFinder",description="The tool searches for job based on the query")

In [17]:
def save_jobs_to_db(jobs,conn,resume_id):
    query = """
    INSERT INTO job_descriptions (resume_id, original_filename,parsed_json)
    VALUES (%s,%s,%s)
    RETURNING id;
    """
    try:
        if conn:
            params = (
                resume_id,
                "API_SEARCH",
                json.dumps(jobs)
                )
            execute_query(conn,query,params)
            return "Job saved to DB"
        else:
            raise Exception("Connection not established")
    except Exception as e:
        return str(e)    

In [18]:
resume_id = ids[-1]
print(resume_id)

2b00559f-245c-4a8d-b90e-be5103499e55


In [19]:
from typing import Optional, List, Literal
from pydantic import BaseModel, Field

class JobDescriptionData(BaseModel):
    """Schema for a single job description entry."""
    
    role: str = Field(..., description="Job title or position name.")
    
    required_skills: List[str] = Field(
        default_factory=list,
        description="List of essential skills required for this role."
    )
    
    required_experience: Optional[str] = Field(
        None,
        description="Experience required for this role (e.g., '3+ years in Python development')."
    )
    
    required_education: Optional[str] = Field(
        None,
        description="Minimum education requirement for the role."
    )
    
    keywords: List[str] = Field(
        default_factory=list,
        description="Important keywords extracted from the job description."
    )
    
    description_text: Optional[str] = Field(
        None,
        description="Full textual job description."
    )
    
    job_employment_type: Optional[
        Literal["Full-time", "Part-time", "Contract", "Internship", "Remote", "Hybrid", "Onsite"]
    ] = Field(
        None,
        description="Employment type for this role."
    )
    
    job_country: Optional[str] = Field(
        None,
        description="Country where the job is located."
    )

    job_apply_link : Optional[str] = Field(
        None,
        description="Link to apply for the job."
    )

class JobDescriptionList(BaseModel):
    """Schema for a collection of job descriptions."""
    
    jobs: List[JobDescriptionData] = Field(
        default_factory=list,
        description="List of job descriptions the candidate can apply for."
    )


In [20]:
job_model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_key=openai_api_key,
    response_format= JobDescriptionList
)

In [21]:
job_description_agent = AssistantAgent(
    name="Job_Description_Agent",
    model_client= model_client,
    system_message="""
    You are a job description agent. You have been provided a tool name JobFinder where you can query job based on the user parsed resume and give him top 5 jobs for him make sure you follow the the given format and give the answer in json format only 
    When using the tool just write query in 2 words like "python developer" or "Machine Developer" not more than that. 
    In the api call result you wil get apply link make sure to add this.
    """,
    tools=[JobFinder],
    reflect_on_tool_use=True
)

In [22]:
job_desc_result = await Console(job_description_agent.run_stream(task=TextMessage(content=f"This is the parsed resume of the candidate\n\n\n {content}",source="user")))
result_2 = job_desc_result.messages[-1].content


---------- TextMessage (user) ----------
This is the parsed resume of the candidate


 {"file_type":"text/plain","name":"John Doe","contact":{"email":"john.doe@example.com","phone":"+1 987 654 3210"},"skills":["Python","SQL","Java","Machine Learning","AWS","Docker","Kubernetes"],"experience":["Software Engineer - ABC Tech (2018 - 2020)","Senior Software Engineer - XYZ Solutions (2020 - Present)"],"education":["B.Tech Computer Science - University of Example (2014 - 2018)"],"certifications":["AWS Certified Developer"]}
---------- ToolCallRequestEvent (Job_Description_Agent) ----------
[FunctionCall(id='fc_9dd1ee52-16af-4efa-b478-657db9201a48', arguments='{"query":"Machine Learning"}', name='JobFinder')]
---------- ToolCallExecutionEvent (Job_Description_Agent) ----------
[FunctionExecutionResult(content='[{\'role\': \'Machine Learning Engineer – ML Ops & Support\', \'company\': \'Aera Technology\', \'employment_type\': \'Full–time\', \'job_location\': \'Pune, Maharashtra\', \'job_salary

In [23]:
result_2 = json.loads(result_2)
result_2 

{'jobs': [{'role': 'Machine Learning Engineer – ML Ops & Support',
   'company': 'Aera Technology',
   'location': 'Pune, Maharashtra',
   'employment_type': 'Full‑time',
   'salary': 'Not disclosed',
   'job_description': 'Aera Technology is seeking a Machine Learning Engineer (Support & Ops) to ensure AI‑powered decision systems run reliably at scale. Responsibilities include monitoring and maintaining ML pipelines and services, building observability tools, supporting LLM/Agentic AI features, developing internal ML tooling, and collaborating with cross‑functional teams. Required skills: 3‑5 years of software engineering/ML Ops experience, strong Python, containerization (Docker, Kubernetes), CI/CD, monitoring tools (Prometheus, Grafana), and model serving frameworks. Familiarity with AWS, Docker, Kubernetes, and Python aligns well with the candidate’s skill set.',
   'job_apply_link': 'https://in.linkedin.com/jobs/view/machine-learning-engineer-%E2%80%93-ml-ops-support-at-aera-techn

In [24]:
save_jobs_to_db(result_2,conn_online_db,resume_id)

'Job saved to DB'

In [25]:
def bring_jobs_data_from_db(conn,resume_id):
    query = """
    SELECT parsed_json FROM job_descriptions WHERE resume_id = %s;
    """
    try:
        if conn:
            params = (
                resume_id,
            )
            result_brought = execute_query(conn_online_db,query,params)
            return result_brought
        else:
            raise Exception("Connection not established")
    except Exception as e:
        return str(e)    

In [26]:
result_3 = bring_jobs_data_from_db(conn_online_db,resume_id)
for i in result_3:
    print(i)

Query Executed
({'jobs': [{'role': 'Machine Learning Engineer – ML Ops & Support', 'salary': 'Not disclosed', 'company': 'Aera Technology', 'location': 'Pune, Maharashtra', 'job_apply_link': 'https://in.linkedin.com/jobs/view/machine-learning-engineer-%E2%80%93-ml-ops-support-at-aera-technology-4285700571?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic', 'employment_type': 'Full‑time', 'job_description': 'Aera Technology is seeking a Machine Learning Engineer (Support & Ops) to ensure AI‑powered decision systems run reliably at scale. Responsibilities include monitoring and maintaining ML pipelines and services, building observability tools, supporting LLM/Agentic AI features, developing internal ML tooling, and collaborating with cross‑functional teams. Required skills: 3‑5 years of software engineering/ML Ops experience, strong Python, containerization (Docker, Kubernetes), CI/CD, monitoring tools (Prometheus, Grafana), and model serving frameworks. Fam

In [27]:
model_client = OpenAIChatCompletionClient(
    base_url="https://api.groq.com/openai/v1", 
    model="openai/gpt-oss-20b", 
    api_key=api_key, 
    model_info={
        "family": "groq",
        "vision": False, 
        "function_calling": True,
        "json_output": False
    }
) 

In [28]:
avaible_job_agent = AssistantAgent(
    name="avaible_job",
    model_client= model_client,
    system_message= """You are a available job agent which get data about jobs the user can do based on his/her resume in json format. You have to show the data beutifully to the user in a detailed way point wise. Dont give any "## ✅ How to Proceed" or "**Tip:** like paragraph just the jobs data."""
)

availeb_job_result = await Console(avaible_job_agent.run_stream(task=TextMessage(content=f"This are the avaible job for user based on its resume\n\n\n {result_3}",source="user")))
result_4 = availeb_job_result.messages[-1].content

---------- TextMessage (user) ----------
This are the avaible job for user based on its resume


 [({'jobs': [{'role': 'Machine Learning Engineer – ML Ops & Support', 'salary': 'Not disclosed', 'company': 'Aera Technology', 'location': 'Pune, Maharashtra', 'job_apply_link': 'https://in.linkedin.com/jobs/view/machine-learning-engineer-%E2%80%93-ml-ops-support-at-aera-technology-4285700571?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic', 'employment_type': 'Full‑time', 'job_description': 'Aera Technology is seeking a Machine Learning Engineer (Support & Ops) to ensure AI‑powered decision systems run reliably at scale. Responsibilities include monitoring and maintaining ML pipelines and services, building observability tools, supporting LLM/Agentic AI features, developing internal ML tooling, and collaborating with cross‑functional teams. Required skills: 3‑5 years of software engineering/ML Ops experience, strong Python, containerization (Docker, Kubernete

In [29]:
ats_model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_key=openai_api_key,
)

In [85]:
ats_scoring_agent = AssistantAgent(
    name="ATS_Scoring_Agent",
    model_client= model_client,
    system_message = """
You are an ATS (Applicant Tracking System) Scoring Agent.

Task:
- Given a parsed resume (JSON) and a list of multiple parsed job descriptions (JSON), calculate ONE overall ATS score representing the candidate’s average fit across all provided roles.

Scoring Method:
1. Compare the resume to each job description using the following categories:
   - Skills Match: 30%
   - Experience Relevance: 25%
   - Education Alignment: 15%
   - Format & Structure: 15%
   - Keyword Optimization: 15%
2. For each category, average the category scores across all jobs.
3. Apply the category weights to the averaged category scores to compute a final weighted total score.
4. This must produce a single overall ATS score for all jobs combined.

Consistency Rules:
- The same resume and job descriptions should always produce the same score (deterministic).
- No randomness or subjective guessing.
- If a category is missing data (e.g., job has no education requirement), base the score only on available information and note it in reasoning.

Benchmark Comparison:
- 85–100: Excellent fit
- 70–84: Good fit
- 50–69: Average fit
- Below 50: Poor fit

Confidence Intervals:
- Confidence is lower if job descriptions vary greatly or have missing details.
- Provide ± percentage confidence interval.

Output:
Return only valid JSON:

{
  "total_score": <integer>,
  "category_scores": {
    "skills_match": <integer>,
    "experience_relevance": <integer>,
    "education_alignment": <integer>,
    "format_structure": <integer>,
    "keyword_optimization": <integer>
  },
  "weighted_breakdown": {
    "skills_match_weighted": <float>,
    "experience_relevance_weighted": <float>,
    "education_alignment_weighted": <float>,
    "format_structure_weighted": <float>,
    "keyword_optimization_weighted": <float>
  },
  "benchmark_comparison": "<string>",
  "confidence_interval": "+/-<integer>%",
  "reasoning": {
    "skills_match": "<explanation>",
    "experience_relevance": "<explanation>",
    "education_alignment": "<explanation>",
    "format_structure": "<explanation>",
    "keyword_optimization": "<explanation>"
  }
}

Guidelines:
- Only use the provided resume and job descriptions.
- Average scores across jobs first, then apply weights.
- The sum of weighted scores must equal the total_score.
- Dont write this ```json just json brackets only "{}"
"""

)

In [86]:
ats_score = await Console(ats_scoring_agent.run_stream(task=TextMessage(content=f"This is parsed resume of the candidate\n\n\n {content} and the sutiable jobs finded using agent\n\n\n {result_3}",source="user")))
ast_result = ats_score.messages[-1].content

---------- TextMessage (user) ----------
This is parsed resume of the candidate


 {"file_type":"text/plain","name":"John Doe","contact":{"email":"john.doe@example.com","phone":"+1 987 654 3210"},"skills":["Python","SQL","Java","Machine Learning","AWS","Docker","Kubernetes"],"experience":["Software Engineer - ABC Tech (2018 - 2020)","Senior Software Engineer - XYZ Solutions (2020 - Present)"],"education":["B.Tech Computer Science - University of Example (2014 - 2018)"],"certifications":["AWS Certified Developer"]} and the sutiable jobs finded using agent


 [({'jobs': [{'role': 'Machine Learning Engineer – ML Ops & Support', 'salary': 'Not disclosed', 'company': 'Aera Technology', 'location': 'Pune, Maharashtra', 'job_apply_link': 'https://in.linkedin.com/jobs/view/machine-learning-engineer-%E2%80%93-ml-ops-support-at-aera-technology-4285700571?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic', 'employment_type': 'Full‑time', 'job_description': 'Aera Techn

In [87]:
result_4 = json.loads(ast_result)
result_4

{'total_score': 80,
 'category_scores': {'skills_match': 63,
  'experience_relevance': 82,
  'education_alignment': 100,
  'format_structure': 90,
  'keyword_optimization': 70},
 'weighted_breakdown': {'skills_match_weighted': 19,
  'experience_relevance_weighted': 21,
  'education_alignment_weighted': 15,
  'format_structure_weighted': 14,
  'keyword_optimization_weighted': 11},
 'benchmark_comparison': 'Good fit',
 'confidence_interval': '+/-5%',
 'reasoning': {'skills_match': 'The candidate’s skill set matches 63% of the key technical requirements across the five job listings, based on the presence of Python, Docker, Kubernetes, AWS, and general machine‑learning terminology.',
  'experience_relevance': 'With over five years of senior software engineering experience in roles that involve ML and cloud services, the candidate’s experience aligns highly (average 82%) with the experience expectations of the positions.',
  'education_alignment': 'The candidate holds a B.Tech in Computer S

In [33]:
def send_ats_score(ats_score,conn,resume_id):
    query = """
    INSERT INTO scores (ats_score,resume_id)
    VALUES (%s,%s)
    """
    try:
        if conn:
            params = (
                json.dumps(ats_score),
                resume_id,
            )
            execute_query(conn,query,params)
            return "Completed"
        else:
            raise Exception("Connection not established")
    except Exception as e:
        return str(e)    

In [34]:
send_ats_score(result_4,conn_online_db,resume_id)

'Completed'

In [35]:
from autogen_core import try_get_known_serializers_for_type


def bring_the_ats_score(conn,resume_id):
    query = """
    SELECT ats_score FROM scores WHERE resume_id = %s
    """
    try:
        if conn:
            params = (resume_id,)
            result = execute_query(conn,query,params)
            return result
        else:
            raise Exception("Connection is not established")
    except Exception as e:
        print(f"Error occured while fetching ATs score for resume_id: {resume_id}, Error: {str(e)}")
        return None

In [36]:
result_5=bring_the_ats_score(conn_online_db,resume_id)

Query Executed


In [37]:
from typing import List
from pydantic import BaseModel,Field

class ImprovementRecommendations(BaseModel):
    """Schema for ATS improvement suggestions."""

    missing_keywords : List[str] = Field(default_factory=list,
    description="List of important keywords missing from the resume.")

    skill_gaps : List[str] = Field(default_factory=list,
    description="List of skills required by the job but missing from the resume.")

    format_recommendations : List[str] = Field(default_factory=list,
    description="Suggestions for improving resume formatting for ATS parsing.")

    content_optimization : List[str] = Field(default_factory=list,
    description="Suggestions for rewriting or restructuring content.")

    priority_actions: List[str] = Field(default_factory=list, description="Top 3 priority actions to improve score the most.")

In [38]:
improvement_model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    api_key = openai_api_key,
    response_format=ImprovementRecommendations
)

In [39]:
improvement_recommendation = AssistantAgent(
    name = "Improvement_Agent",
    model_client=model_client,
    system_message="""You are an ATS Improvement Recommendation Agent.

Your role:
- Compare a candidate's parsed resume JSON against one or more parsed job description JSONs.
- Use the ATS scoring output (if provided) to identify weak scoring categories.
- Provide detailed, actionable improvements to increase the candidate's score and job fit.

Analysis Steps:
1. **Keyword Match**:
   - Identify missing keywords from the job description(s) that are relevant to the candidate’s background.
   - Suggest exact phrases or skills that should be added to the resume.

2. **Skill Gap Analysis**:
   - Compare required and preferred skills in the job description(s) with the resume’s skill list.
   - Recommend upskilling paths or certifications for missing but important skills.

3. **Format & Structure Recommendations**:
   - Suggest resume formatting improvements for ATS parsing (e.g., section ordering, avoiding tables/images, consistent headings).

4. **Content Optimization**:
   - Recommend restructuring bullet points to be more impactful and keyword-rich.
   - Suggest quantifying achievements (e.g., “Increased sales by 20%” instead of “Helped improve sales”).

Output:
Return only valid JSON in the following format:

Guidelines:
- Be specific — provide examples for improvements.
- Only suggest changes relevant to the job description(s).
- If nothing is missing in a category, return an empty list for that category.
- Do not rewrite the resume — only suggest improvements."""
)

In [40]:
improve_agent_result = await Console(improvement_recommendation.run_stream(task=TextMessage(content=f"The candidate parsed resume is\n\n\n {content} and the job descriptions are \n\n\n {result_3} and the ats score {result_4}",source="user")))

improve_result = improve_agent_result.messages[-1].content

---------- TextMessage (user) ----------
The candidate parsed resume is


 {"file_type":"text/plain","name":"John Doe","contact":{"email":"john.doe@example.com","phone":"+1 987 654 3210"},"skills":["Python","SQL","Java","Machine Learning","AWS","Docker","Kubernetes"],"experience":["Software Engineer - ABC Tech (2018 - 2020)","Senior Software Engineer - XYZ Solutions (2020 - Present)"],"education":["B.Tech Computer Science - University of Example (2014 - 2018)"],"certifications":["AWS Certified Developer"]} and the job descriptions are 


 [({'jobs': [{'role': 'Machine Learning Engineer – ML Ops & Support', 'salary': 'Not disclosed', 'company': 'Aera Technology', 'location': 'Pune, Maharashtra', 'job_apply_link': 'https://in.linkedin.com/jobs/view/machine-learning-engineer-%E2%80%93-ml-ops-support-at-aera-technology-4285700571?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic', 'employment_type': 'Full‑time', 'job_description': 'Aera Technology is seeking a 

In [41]:
content 

'{"file_type":"text/plain","name":"John Doe","contact":{"email":"john.doe@example.com","phone":"+1 987 654 3210"},"skills":["Python","SQL","Java","Machine Learning","AWS","Docker","Kubernetes"],"experience":["Software Engineer - ABC Tech (2018 - 2020)","Senior Software Engineer - XYZ Solutions (2020 - Present)"],"education":["B.Tech Computer Science - University of Example (2014 - 2018)"],"certifications":["AWS Certified Developer"]}'

In [42]:
result_3

[({'jobs': [{'role': 'Machine Learning Engineer – ML Ops & Support',
     'salary': 'Not disclosed',
     'company': 'Aera Technology',
     'location': 'Pune, Maharashtra',
     'job_apply_link': 'https://in.linkedin.com/jobs/view/machine-learning-engineer-%E2%80%93-ml-ops-support-at-aera-technology-4285700571?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic',
     'employment_type': 'Full‑time',
     'job_description': 'Aera Technology is seeking a Machine Learning Engineer (Support & Ops) to ensure AI‑powered decision systems run reliably at scale. Responsibilities include monitoring and maintaining ML pipelines and services, building observability tools, supporting LLM/Agentic AI features, developing internal ML tooling, and collaborating with cross‑functional teams. Required skills: 3‑5 years of software engineering/ML Ops experience, strong Python, containerization (Docker, Kubernetes), CI/CD, monitoring tools (Prometheus, Grafana), and model serving

In [43]:
result_4

'**Machine Learning Engineer – ML Ops & Support**  \n- **Company:** Aera Technology – Pune, Maharashtra  \n- **Employment type:** Full‑time  \n- **Salary:** Not disclosed  \n- **Apply link:** <https://in.linkedin.com/jobs/view/machine-learning-engineer-%E2%80%93-ml-ops-support-at-aera-technology-4285700571?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic>  \n- **Key responsibilities:**  \n  - Monitor and maintain ML pipelines and services.  \n  - Build observability tools and support LLM/Agentic AI features.  \n  - Develop internal ML tooling and collaborate with cross‑functional teams.  \n- **Required skills:** 3‑5\u202fyrs of software engineering/ML Ops, Python, Docker, Kubernetes, CI/CD, Prometheus/Grafana, AWS.  \n\n---\n\n**AI/ML Solutions Architect – GenAI & Cloud (BFSI Focus)**  \n- **Company:** vueverse. – Guwahati, Assam  \n- **Employment type:** Full‑time  \n- **Salary:** Not disclosed  \n- **Apply link:** <https://en-in.whatjobs.com/jobs/machine

In [44]:
json.dumps(result_5)

'[["**Machine Learning Engineer \\u2013 ML Ops & Support**  \\n- **Company:** Aera Technology \\u2013 Pune, Maharashtra  \\n- **Employment type:** Full\\u2011time  \\n- **Salary:** Not disclosed  \\n- **Apply link:** <https://in.linkedin.com/jobs/view/machine-learning-engineer-%E2%80%93-ml-ops-support-at-aera-technology-4285700571?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic>  \\n- **Key responsibilities:**  \\n  - Monitor and maintain ML pipelines and services.  \\n  - Build observability tools and support LLM/Agentic AI features.  \\n  - Develop internal ML tooling and collaborate with cross\\u2011functional teams.  \\n- **Required skills:** 3\\u20115\\u202fyrs of software engineering/ML Ops, Python, Docker, Kubernetes, CI/CD, Prometheus/Grafana, AWS.  \\n\\n---\\n\\n**AI/ML Solutions Architect \\u2013 GenAI & Cloud (BFSI Focus)**  \\n- **Company:** vueverse. \\u2013 Guwahati, Assam  \\n- **Employment type:** Full\\u2011time  \\n- **Salary:** Not dis

In [45]:
def send_improve_to_score(result,conn,resume_id):
    query = """
    UPDATE scores SET recommendations = %s where resume_id = %s;
    """
    try:
        if conn:
            params = (json.dumps(result),resume_id,)
            execute_query(conn,query,params)
            return "Completed"
        else:
            raise Exception("Connection is not established")
    except Exception as e:
        print(f"Error occured while fetching ATs score for resume_id: {resume_id}, Error: {str(e)}")
        return None

In [46]:
send_improve_to_score(result_5,conn_online_db,resume_id)

'Completed'

In [47]:
groq_api_key = os.getenv('GROQ_API_KEY')

In [48]:
groq_client = OpenAIChatCompletionClient(
    base_url="https://api.groq.com/openai/v1",  # Groq's OpenAI-compatible endpoint
    model="openai/gpt-oss-20b",  # Example model, you can change this to other available Groq models
    api_key=groq_api_key,  # Your Groq API key
    model_info={
        "family": "groq",
        "vision": False,  # Most Groq models don't support vision
        "function_calling": True,  # Check Groq's documentation for specific model capabilities
        "json_output": False,
        "multiple_system_messages": True,
    }
)

In [49]:
summary_agent = AssistantAgent(
    name="summarizer_agent",
    model_client=groq_client,
    system_message="""
You are an ATS Summary Agent.

Your job:
- Take structured JSON from the ATS scoring, improvement recommendations, and optionally the parsed job description.
- Convert these into a clear, concise, and encouraging detailed summary for the candidate.
- The goal is to help the candidate understand:
  1. Their overall ATS score and what it means.
  2. Key strengths.
  3. Areas needing improvement.
  4. Concrete next steps they can take.

Guidelines:
- Be professional but friendly.
- Avoid overly technical ATS terms unless explained simply.
- Keep paragraphs short and scannable.
- Use bullet points for recommendations.

Example Output Format:

"Your overall ATS score is **72/100**, which is considered a **Good Fit** for the jobs you can target.

**Strengths:**
- Strong alignment with required experience (8+ years relevant).
- Education fully matches or exceeds requirements.
- Resume structure is clean and ATS-friendly.

**Areas for Improvement:**
- Missing several high-value keywords: AWS Lambda, API Gateway, Terraform.
- Format could benefit from more action verbs and quantifiable results.
- Some job-specific skills not highlighted enough.

**Next Steps:**
1. Add missing keywords naturally into your work experience section.
2. Emphasize AWS-related projects with measurable impact.
3. Include a dedicated skills section listing relevant technologies.

With these adjustments, your score could increase by 10–15 points, moving you into the Excellent Fit range."

Instructions:
- Do not return raw JSON — return plain text suitable for displaying directly to the user.
- Pull information from all provided JSONs.
"""
)

In [50]:
from asyncio import Task
from typing import Text


summary_agent_result = await Console(summary_agent.run_stream(task=TextMessage(content=f"The candidate parsed resume is\n\n\n {content} and the job descriptions are \n\n\n {result_3} and the ats score {result_4}",source="user")))

summary_result = summary_agent_result.messages[-1].content

---------- TextMessage (user) ----------
The candidate parsed resume is


 {"file_type":"text/plain","name":"John Doe","contact":{"email":"john.doe@example.com","phone":"+1 987 654 3210"},"skills":["Python","SQL","Java","Machine Learning","AWS","Docker","Kubernetes"],"experience":["Software Engineer - ABC Tech (2018 - 2020)","Senior Software Engineer - XYZ Solutions (2020 - Present)"],"education":["B.Tech Computer Science - University of Example (2014 - 2018)"],"certifications":["AWS Certified Developer"]} and the job descriptions are 


 [({'jobs': [{'role': 'Machine Learning Engineer – ML Ops & Support', 'salary': 'Not disclosed', 'company': 'Aera Technology', 'location': 'Pune, Maharashtra', 'job_apply_link': 'https://in.linkedin.com/jobs/view/machine-learning-engineer-%E2%80%93-ml-ops-support-at-aera-technology-4285700571?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic', 'employment_type': 'Full‑time', 'job_description': 'Aera Technology is seeking a 

## Creating RAG

In [51]:
from autogen_core.memory import MemoryContent,MemoryMimeType
from autogen_ext.memory.mem0 import Mem0Memory

In [ ]:
import json

mem0_memory = Mem0Memory(
    is_cloud=True,
    api_key = "m0-QXXhimSTpF0BvB3Gf3COiZIG85WknDgFbrkQDBWd",
    limit=3,
    user_id="ats_id"
)

openai_model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=openai_api_key
)

file_path = "realistic_rag_benchmarks.jsonl"

with open(file_path,"r",encoding="utf-8") as file:
    for line in file:
        entry = json.loads(line.strip())
        print(entry)

        await mem0_memory.add(
            MemoryContent(
                content=entry['content'],
                mime_type=MemoryMimeType.TEXT,
                metadata=entry['metadata']
            )
        )






{'content': 'In Tech, the industry standard for skills match is 85%.', 'metadata': {'category': 'skills_match', 'industry': 'Tech', 'type': 'best_practice', 'industry_standard': 85, 'unit': 'percent', 'common_gaps': ['Missing core frameworks/tools', 'Outdated technical skills', 'No mention of in-demand software'], 'optimization_tips': ['List all relevant tools and frameworks', 'Highlight in-demand skills in top sections', 'Align skills with job description wording']}}
{'content': 'In Tech, the industry standard for experience relevance is 90%.', 'metadata': {'category': 'experience_relevance', 'industry': 'Tech', 'type': 'best_practice', 'industry_standard': 90, 'unit': 'percent', 'common_gaps': ['No quantified results', 'Missing role-specific projects', 'Unclear responsibilities'], 'optimization_tips': ['Add measurable results to achievements', 'Focus on industry-specific experience', 'Match job titles to posting']}}
{'content': 'In Tech, the industry standard for education alignment 

In [52]:
openai_model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    api_key=openai_api_key,
    model_info={
        "multiple_system_messages":True,
        "vision":False,
        "function_calling":True,
        "json_output":False,
        "family":"gpt-4o"
    }
)
mem0_memory = Mem0Memory(
    is_cloud=True,
    api_key = "m0-QXXhimSTpF0BvB3Gf3COiZIG85WknDgFbrkQDBWd",
    limit=3,
    user_id="ats_id"
)

c:\Users\mkuma\OneDrive\Desktop\MEOW MEOW 1.0\.venv\Lib\site-packages\autogen_ext\models\openai\_openai_client.py:453: UserWarning: Missing required field 'structured_output' in ModelInfo. This field will be required in a future version of AutoGen.
  validate_model_info(self._model_info)


In [53]:
assistant = AssistantAgent(
    name = "assistants",
    model_client= groq_client,
    memory = [mem0_memory],
    system_message="""You are a Visualization Agent in an ATS scoring system. Your responsibilities are:

Task Overview:
1. Retrieve and Compare Data
   - Receive ATS scoring data from user input.
   - Retrieve corresponding industry benchmarks from memory.

2. Generate Visualizations using Python
   - Bar Chart: Compare user scores vs. industry benchmarks for each category.
   - Pie Chart: Show weighted score contributions of each category.
   - Radar Chart: Show category performance distribution.
   - I want them to be combined in a same png file with name "combine_chart.png".
   - At last print something like this "print("Charts generated")"

3. Code Generation Requirements
   - Write the Python code using matplotlib and seaborn.
   - Include code to save the generated images in the same directory.
   - Structure the code in a single code block.
   - You have to write the code in the example format:
     ```python
     code = '''
         print("Hello World")
     '''
     with open('temp/sample.png', 'w') as f:
         f.write(code)
         print("Code saved successfully in solution.py")
     ```

4. Response Structure
   - Step 1: Explain your plan to solve the task.
   - Step 2: Provide the Python code inside a single code block.
   - Step 3: After execution, send the code to visualizer agent make sure the code only have matplotlib, seaborn and numpy import.

Important Notes:
- Always ensure the charts accurately reflect user scores vs. industry benchmarks.
- Ensure the pie chart correctly uses weighted contributions.
- Radar chart axes must correspond to each scoring category.
- Do not split code into multiple blocks.
- Follow the template strictly for saving the code."""
)

# stream = assistant.run_stream(task=f"The ats score of the user resume is {result_4}")
# await Console(stream)

In [54]:
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor

In [55]:
visualization_agent = AssistantAgent(
    name="Visulizer_Agent",
    model_client=groq_client,
    system_message="The assistant will research and give you the code for creating the graph you have to just send the python scripts to the code executor agent provided by the assistant if any error occurs try to fix it and give the code again to the code executor agent. Stricltly only give the ```python code to the code executor agent. You dont have use import os or anything just give the code what you have received from the assistant agent. "
)

In [56]:
docker = DockerCommandLineCodeExecutor(
    image="python-with-matplotlib-and-seaborn",
    work_dir = "temp",
    timeout=120
)

In [57]:
from autogen_agentchat.agents import CodeExecutorAgent
code_executor_agent = CodeExecutorAgent(
    name="Code_Executor_Agent",
    code_executor=docker
)

In [59]:
await docker.start()

In [61]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination

sample_team = RoundRobinGroupChat(
    participants=[assistant],
    termination_condition=TextMentionTermination("STOP"),
    max_turns = 5
)
async def main():
    task = f"The ATS score of the candidate resume is {result_4}"
    await Console(sample_team.run_stream(task=task))
    if assistant.name:
        print("Yes assistant is called")

await main()

---------- TextMessage (user) ----------
The ATS score of the candidate resume is **Machine Learning Engineer – ML Ops & Support**  
- **Company:** Aera Technology – Pune, Maharashtra  
- **Employment type:** Full‑time  
- **Salary:** Not disclosed  
- **Apply link:** <https://in.linkedin.com/jobs/view/machine-learning-engineer-%E2%80%93-ml-ops-support-at-aera-technology-4285700571?utm_campaign=google_jobs_apply&utm_source=google_jobs_apply&utm_medium=organic>  
- **Key responsibilities:**  
  - Monitor and maintain ML pipelines and services.  
  - Build observability tools and support LLM/Agentic AI features.  
  - Develop internal ML tooling and collaborate with cross‑functional teams.  
- **Required skills:** 3‑5 yrs of software engineering/ML Ops, Python, Docker, Kubernetes, CI/CD, Prometheus/Grafana, AWS.  

---

**AI/ML Solutions Architect – GenAI & Cloud (BFSI Focus)**  
- **Company:** vueverse. – Guwahati, Assam  
- **Employment type:** Full‑time  
- **Salary:** Not disclosed  

### Agent 1

In [ ]:
extracted_text_json = AssistantAgent(
    name="json_extractor",
    model_client= json_model_client,
    system_message="""You are a resume parser agent when you are given a resume text, you will extract the information from the resume and return it in a json format. In the extracted text there is also file_type written as starting""")

### Agent 2

In [141]:
job_description_agent = AssistantAgent(
    name="Job_Description_Agent",
    model_client= model_client,
    system_message="""
    You are a job description agent. You have been provided a tool name JobFinder where you can query job based on the user parsed resume and give him top 5 jobs for him make sure you follow the the given format and give the answer in json format only 
    When using the tool just write query in 2 words like "python developer" or "Machine Developer" not more than that. 
    In the api call result you wil get apply link make sure to add this.
    """,
    tools=[JobFinder],
    reflect_on_tool_use=True
)

### Agent 3

In [142]:
avaible_job_agent = AssistantAgent(
    name="avaible_job",
    model_client= model_client,
    system_message= """You are a available job agent which get data about jobs the user can do based on his/her resume in json format. You have to show the data beutifully to the user in a detailed way point wise. Dont give any "## ✅ How to Proceed" or "**Tip:** like paragraph just the jobs data."""
)

### Agent 4

In [143]:
ats_scoring_agent = AssistantAgent(
    name="ATS_Scoring_Agent",
    model_client= model_client,
    system_message = """
You are an ATS (Applicant Tracking System) Scoring Agent.

Task:
- Given a parsed resume (JSON) and a list of multiple parsed job descriptions (JSON), calculate ONE overall ATS score representing the candidate’s average fit across all provided roles.

Scoring Method:
1. Compare the resume to each job description using the following categories:
   - Skills Match: 30%
   - Experience Relevance: 25%
   - Education Alignment: 15%
   - Format & Structure: 15%
   - Keyword Optimization: 15%
2. For each category, average the category scores across all jobs.
3. Apply the category weights to the averaged category scores to compute a final weighted total score.
4. This must produce a single overall ATS score for all jobs combined.

Consistency Rules:
- The same resume and job descriptions should always produce the same score (deterministic).
- No randomness or subjective guessing.
- If a category is missing data (e.g., job has no education requirement), base the score only on available information and note it in reasoning.

Benchmark Comparison:
- 85–100: Excellent fit
- 70–84: Good fit
- 50–69: Average fit
- Below 50: Poor fit

Confidence Intervals:
- Confidence is lower if job descriptions vary greatly or have missing details.
- Provide ± percentage confidence interval.

Output:
Return only valid JSON:

{
  "total_score": <integer>,
  "category_scores": {
    "skills_match": <integer>,
    "experience_relevance": <integer>,
    "education_alignment": <integer>,
    "format_structure": <integer>,
    "keyword_optimization": <integer>
  },
  "weighted_breakdown": {
    "skills_match_weighted": <float>,
    "experience_relevance_weighted": <float>,
    "education_alignment_weighted": <float>,
    "format_structure_weighted": <float>,
    "keyword_optimization_weighted": <float>
  },
  "benchmark_comparison": "<string>",
  "confidence_interval": "+/-<integer>%",
  "reasoning": {
    "skills_match": "<explanation>",
    "experience_relevance": "<explanation>",
    "education_alignment": "<explanation>",
    "format_structure": "<explanation>",
    "keyword_optimization": "<explanation>"
  }
}

Guidelines:
- Only use the provided resume and job descriptions.
- Average scores across jobs first, then apply weights.
- The sum of weighted scores must equal the total_score.
"""

)

### Agent 5

In [ ]:
assistant = AssistantAgent(
    name = "assistants",
    model_client= groq_client,
    memory = [mem0_memory],
    system_message="""You are a Visualization Agent in an ATS scoring system. Your responsibilities are:

Task Overview:
Retrieve and Compare Data
   - Receive ATS scoring data from user input.
   - Retrieve corresponding industry benchmarks from memory.
"""
)

### Agent 6

In [254]:
visualization_agent = AssistantAgent(
    name="Visualizer_Agent",
    model_client=groq_client,
    system_message="""
    Task Instructions:

    1. You will receive the required data from the previous agent.
       feed the data in the python code below.
    2. Dont try to write any other than the code given below just fill the values and give it further to next agent. 

   ```python
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# ------------------------------------------------------------------
# 1. Data placeholders (Agent should replace with actual values)
# ------------------------------------------------------------------
user_scores = {
    "total_score": <TOTAL_SCORE>,  # e.g., 82
    "category_scores": {
        "skills_match": <SKILLS_MATCH_SCORE>,                # e.g., 62
        "experience_relevance": <EXPERIENCE_RELEVANCE_SCORE>, # e.g., 100
        "education_alignment": <EDUCATION_ALIGNMENT_SCORE>,   # e.g., 100
        "format_structure": <FORMAT_STRUCTURE_SCORE>,         # e.g., 95
        "keyword_optimization": <KEYWORD_OPTIMIZATION_SCORE>  # e.g., 62
    },
    "weighted_breakdown": {
        "skills_match_weighted": <SKILLS_MATCH_WEIGHTED>,                # e.g., 19.0
        "experience_relevance_weighted": <EXPERIENCE_RELEVANCE_WEIGHTED>, # e.g., 25.0
        "education_alignment_weighted": <EDUCATION_ALIGNMENT_WEIGHTED>,   # e.g., 15.0
        "format_structure_weighted": <FORMAT_STRUCTURE_WEIGHTED>,         # e.g., 14.0
        "keyword_optimization_weighted": <KEYWORD_OPTIMIZATION_WEIGHTED>  # e.g., 9.0
    }
}

# Industry benchmarks placeholders
industry_benchmarks = {
    "skills_match": <BENCHMARK_SKILLS_MATCH>,              
    "experience_relevance": <BENCHMARK_EXPERIENCE_RELEVANCE>,
    "education_alignment": <BENCHMARK_EDUCATION_ALIGNMENT>, 
    "keyword_optimization": <BENCHMARK_KEYWORD_OPTIMIZATION>
}

# ------------------------------------------------------------------
# 2. Create a single figure with 3 subplots
# ------------------------------------------------------------------
fig = plt.figure(figsize=(15, 5))
sns.set_style("whitegrid")

# ---------------------- Bar Chart ----------------------
ax1 = fig.add_subplot(1, 3, 1)
categories = [
    "skills_match",
    "experience_relevance",
    "education_alignment",
    "keyword_optimization"
]

candidate_vals = [user_scores["category_scores"][c] for c in categories]
benchmark_vals = [industry_benchmarks[c] for c in categories]

bar_width = 0.35
x = np.arange(len(categories))
ax1.bar(x - bar_width/2, candidate_vals, width=bar_width, label="Candidate", color="#4c72b0")
ax1.bar(x + bar_width/2, benchmark_vals, width=bar_width, label="Industry Benchmark", color="#c44e52")
ax1.set_xticks(x)
ax1.set_xticklabels([c.replace('_', ' ').title() for c in categories], rotation=45, ha="right")
ax1.set_ylabel("Score (%)")
ax1.set_title("Candidate vs. Industry Benchmarks")
ax1.set_ylim(0, 110)
ax1.legend()

# ---------------------- Pie Chart ----------------------
ax2 = fig.add_subplot(1, 3, 2)
labels = [
    "Skills Match",
    "Experience Relevance",
    "Education Alignment",
    "Format & Structure",
    "Keyword Optimization"
]
weights = [
    user_scores["weighted_breakdown"]["skills_match_weighted"],
    user_scores["weighted_breakdown"]["experience_relevance_weighted"],
    user_scores["weighted_breakdown"]["education_alignment_weighted"],
    user_scores["weighted_breakdown"]["format_structure_weighted"],
    user_scores["weighted_breakdown"]["keyword_optimization_weighted"]
]
ax2.pie(weights, labels=labels, autopct='%1.1f%%', startangle=140, colors=sns.color_palette("pastel"))
ax2.set_title("Weighted Score Contribution")

# ---------------------- Radar Chart ----------------------
ax3 = fig.add_subplot(1, 3, 3, polar=True)
radar_labels = [
    "Skills Match",
    "Experience Relevance",
    "Education Alignment",
    "Format & Structure",
    "Keyword Optimization"
]
radar_vals = [
    user_scores["category_scores"]["skills_match"],
    user_scores["category_scores"]["experience_relevance"],
    user_scores["category_scores"]["education_alignment"],
    user_scores["category_scores"]["format_structure"],
    user_scores["category_scores"]["keyword_optimization"]
]
values = radar_vals + [radar_vals[0]]
angles = np.linspace(0, 2 * np.pi, len(radar_labels), endpoint=False).tolist()
angles += angles[:1]
ax3.plot(angles, values, 'o-', linewidth=2, color="#1f77b4", label="Candidate")
ax3.fill(angles, values, color="#1f77b4", alpha=0.15)
ax3.set_thetagrids(np.degrees(angles[:-1]), radar_labels)
ax3.set_ylim(0, 110)
ax3.set_title("Performance Radar")
ax3.grid(True)

plt.tight_layout()
plt.savefig("combined_charts.png", dpi=300)
plt.close()

print("Combined chart saved as combined_charts.png")

```
    """
)

### Agent 7

In [255]:
from autogen_agentchat.agents import CodeExecutorAgent
code_executor_agent = CodeExecutorAgent(
    name="Code_Executor_Agent",
    code_executor=docker
)

In [256]:
from autogen_agentchat.agents import SocietyOfMindAgent
inner_team = RoundRobinGroupChat(participants=[visualization_agent,code_executor_agent],termination_condition=TextMentionTermination("STOP"),max_turns = 2)
society_of_mind_agent = SocietyOfMindAgent(name="Sociey_of_mind_agent",team=inner_team,model_client=model_client)

In [90]:
final_team = RoundRobinGroupChat(
    participants=[extracted_text_json,job_description_agent,ats_scoring_agent,improvement_recommendation,assistant,visualization_agent,code_executor_agent,summary_agent],
    termination_condition=TextMentionTermination("STOP"),
    max_turns = 10
)
task = """
'John Doe\nEmail: john.doe@example.com\nPhone: +1 987 654 3210\n\nProfessional Summary:\nHighly motivated software engineer with 4 years of experience in backend and cloud technologies.\n\nSkills:\nPython, SQL, Java, Machine Learning, AWS, Docker, Kubernetes\n\nExperience:\nSoftware Engineer - ABC Tech (2018 - 2020)\nSenior Software Engineer - XYZ Solutions (2020 - Present)\n\nEducation:\nB.Tech Computer Science - University of Example (2014 - 2018)\n\nCertifications:\nAWS Certified Developer'
"""

async for message in final_team.run_stream(task=task):
            if isinstance(message, TextMessage):
                msg = f"{message.source}: {message.content}"
                print(msg)
# task = """
# 'John Doe\nEmail: john.doe@example.com\nPhone: +1 987 654 3210\n\nProfessional Summary:\nHighly motivated software engineer with 4 years of experience in backend and cloud technologies.\n\nSkills:\nPython, SQL, Java, Machine Learning, AWS, Docker, Kubernetes\n\nExperience:\nSoftware Engineer - ABC Tech (2018 - 2020)\nSenior Software Engineer - XYZ Solutions (2020 - Present)\n\nEducation:\nB.Tech Computer Science - University of Example (2014 - 2018)\n\nCertifications:\nAWS Certified Developer'
# """
#     result = await Console(final_team.run_stream(task=TextMessage(content=task,source="user")))
#     last_message = result.messages[-1].content
#     if extracted_text_json.name:
#         save_resume_to_db(conn_online_db,last_message)
#         print("Sended to db")
#     elif job_description_agent.name:
#         save_jobs_to_db(last_message,conn_online_db,resume_id)
#         print("Sended to db")
#     elif ats_scoring_agent.name:
#         send_ats_score(last_message,conn_online_db,resume_id)
#         print("Sended to db")
#     elif improvement_recommendation.name:
#         send_improve_to_score(last_message,conn_online_db,resume_id)
#         print("Sended to db")

    

user: 
'John Doe
Email: john.doe@example.com
Phone: +1 987 654 3210

Professional Summary:
Highly motivated software engineer with 4 years of experience in backend and cloud technologies.

Skills:
Python, SQL, Java, Machine Learning, AWS, Docker, Kubernetes

Experience:
Software Engineer - ABC Tech (2018 - 2020)
Senior Software Engineer - XYZ Solutions (2020 - Present)

Education:
B.Tech Computer Science - University of Example (2014 - 2018)

Certifications:
AWS Certified Developer'

json_extractor: {"file_type":null,"name":"John Doe","contact":{"email":"john.doe@example.com","phone":"+1 987 654 3210"},"skills":["Python","SQL","Java","Machine Learning","AWS","Docker","Kubernetes"],"experience":["Software Engineer - ABC Tech (2018 - 2020)","Senior Software Engineer - XYZ Solutions (2020 - Present)"],"education":["B.Tech Computer Science - University of Example (2014 - 2018)"],"certifications":["AWS Certified Developer"]}
Job_Description_Agent: {
  "jobs": [
    {
      "role": "Senior J

ERROR:autogen_core:Error processing publish message for Job_Description_Agent_87a21c5d-7340-4888-9d5d-78f24cc0c8ec/87a21c5d-7340-4888-9d5d-78f24cc0c8ec
Traceback (most recent call last):
  File "c:\Users\mkuma\OneDrive\Desktop\MEOW MEOW 1.0\.venv\Lib\site-packages\autogen_core\_single_threaded_agent_runtime.py", line 606, in _on_message
    return await agent.on_message(
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mkuma\OneDrive\Desktop\MEOW MEOW 1.0\.venv\Lib\site-packages\autogen_core\_base_agent.py", line 119, in on_message
    return await self.on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mkuma\OneDrive\Desktop\MEOW MEOW 1.0\.venv\Lib\site-packages\autogen_agentchat\teams\_group_chat\_sequential_routed_agent.py", line 67, in on_message_impl
    return await super().on_message_impl(message, ctx)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\mkuma\OneDrive\Desktop\MEOW MEOW 1.0\.venv\Lib\site-pack

RuntimeError: RuntimeError: Reflect on tool use produced no valid text response.
Traceback:
Traceback (most recent call last):

  File "c:\Users\mkuma\OneDrive\Desktop\MEOW MEOW 1.0\.venv\Lib\site-packages\autogen_agentchat\teams\_group_chat\_chat_agent_container.py", line 133, in handle_request
    async for msg in self._agent.on_messages_stream(self._message_buffer, ctx.cancellation_token):

  File "c:\Users\mkuma\OneDrive\Desktop\MEOW MEOW 1.0\.venv\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py", line 989, in on_messages_stream
    async for output_event in self._process_model_result(

  File "c:\Users\mkuma\OneDrive\Desktop\MEOW MEOW 1.0\.venv\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py", line 1299, in _process_model_result
    async for reflection_response in cls._reflect_on_tool_use_flow(

  File "c:\Users\mkuma\OneDrive\Desktop\MEOW MEOW 1.0\.venv\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py", line 1454, in _reflect_on_tool_u

In [71]:
await main()

---------- TextMessage (user) ----------

'John Doe
Email: john.doe@example.com
Phone: +1 987 654 3210

Professional Summary:
Highly motivated software engineer with 4 years of experience in backend and cloud technologies.

Skills:
Python, SQL, Java, Machine Learning, AWS, Docker, Kubernetes

Experience:
Software Engineer - ABC Tech (2018 - 2020)
Senior Software Engineer - XYZ Solutions (2020 - Present)

Education:
B.Tech Computer Science - University of Example (2014 - 2018)

Certifications:
AWS Certified Developer'



---------- TextMessage (json_extractor) ----------
{"file_type":null,"name":"John Doe","contact":{"email":"john.doe@example.com","phone":"+1 987 654 3210"},"skills":["Python","SQL","Java","Machine Learning","AWS","Docker","Kubernetes"],"experience":["Software Engineer - ABC Tech (2018 - 2020)","Senior Software Engineer - XYZ Solutions (2020 - Present)"],"education":["B.Tech Computer Science - University of Example (2014 - 2018)"],"certifications":["AWS Certified Developer"]}
---------- ToolCallRequestEvent (Job_Description_Agent) ----------
[FunctionCall(id='fc_2ad85afa-b12a-4631-806e-ce14a3a15a60', arguments='{"query":"Python developer"}', name='JobFinder')]
---------- ToolCallExecutionEvent (Job_Description_Agent) ----------
[FunctionExecutionResult(content='[{\'role\': \'Lead Software Engineer - Python\', \'company\': \'EPAM Systems\', \'employment_type\': \'Full–time\', \'job_location\': \'India\', \'job_salary\': \'None - None\', \'job_description\': \'We are seeking a highly expe

TypeError: string indices must be integers, not 'str'

In [63]:
extracted_text_json.name

'json_extractor'

In [ ]:
task = """
'John Doe\nEmail: john.doe@example.com\nPhone: +1 987 654 3210\n\nProfessional Summary:\nHighly motivated software engineer with 4 years of experience in backend and cloud technologies.\n\nSkills:\nPython, SQL, Java, Machine Learning, AWS, Docker, Kubernetes\n\nExperience:\nSoftware Engineer - ABC Tech (2018 - 2020)\nSenior Software Engineer - XYZ Solutions (2020 - Present)\n\nEducation:\nB.Tech Computer Science - University of Example (2014 - 2018)\n\nCertifications:\nAWS Certified Developer'
"""
await Console(final_team.run_stream(task=TextMessage(content=task,source="user")))

---------- TextMessage (user) ----------

'John Doe
Email: john.doe@example.com
Phone: +1 987 654 3210

Professional Summary:
Highly motivated software engineer with 4 years of experience in backend and cloud technologies.

Skills:
Python, SQL, Java, Machine Learning, AWS, Docker, Kubernetes

Experience:
Software Engineer - ABC Tech (2018 - 2020)
Senior Software Engineer - XYZ Solutions (2020 - Present)

Education:
B.Tech Computer Science - University of Example (2014 - 2018)

Certifications:
AWS Certified Developer'



c:\Users\mkuma\OneDrive\Desktop\MEOW MEOW 1.0\.venv\Lib\site-packages\autogen_ext\models\openai\_openai_client.py:660: DeprecationWarning: Using response_format to specify the BaseModel for structured output type will be deprecated. Use json_output in create and create_stream instead.
  create_params = self._process_create_args(


---------- TextMessage (json_extractor) ----------
{"file_type":null,"name":"John Doe","contact":{"email":"john.doe@example.com","phone":"+1 987 654 3210"},"skills":["Python","SQL","Java","Machine Learning","AWS","Docker","Kubernetes"],"experience":["Software Engineer - ABC Tech (2018 - 2020)","Senior Software Engineer - XYZ Solutions (2020 - Present)"],"education":["B.Tech Computer Science - University of Example (2014 - 2018)"],"certifications":["AWS Certified Developer"]}
---------- ToolCallRequestEvent (Job_Description_Agent) ----------
[FunctionCall(id='fc_2cd2c2f6-a20a-490a-9dba-40b118a53d94', arguments='{"query":"software engineer"}', name='JobFinder')]
---------- ToolCallExecutionEvent (Job_Description_Agent) ----------
[FunctionExecutionResult(content='[{\'role\': \'System Software Engineer - Java Runtimes, Tooling and Ecosystem\', \'company\': \'Canonical\', \'employment_type\': \'Full–time\', \'job_location\': \'India\', \'job_salary\': \'None - None\', \'job_description\': 

TaskResult(messages=[TextMessage(id='3dd7a63c-82a0-4d4b-ac19-e8adb2155c74', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 14, 11, 40, 28, 353744, tzinfo=datetime.timezone.utc), content="\n'John Doe\nEmail: john.doe@example.com\nPhone: +1 987 654 3210\n\nProfessional Summary:\nHighly motivated software engineer with 4 years of experience in backend and cloud technologies.\n\nSkills:\nPython, SQL, Java, Machine Learning, AWS, Docker, Kubernetes\n\nExperience:\nSoftware Engineer - ABC Tech (2018 - 2020)\nSenior Software Engineer - XYZ Solutions (2020 - Present)\n\nEducation:\nB.Tech Computer Science - University of Example (2014 - 2018)\n\nCertifications:\nAWS Certified Developer'\n", type='TextMessage'), TextMessage(id='ba6b2985-015a-4d74-849e-6410f18bbe33', source='json_extractor', models_usage=RequestUsage(prompt_tokens=403, completion_tokens=110), metadata={}, created_at=datetime.datetime(2025, 8, 14, 11, 40, 31, 728129, tzinfo=datetime.timezone.